# CNN Transfer Learning on Cats-Dogs Classification

## CIML Summer Institute, UC San Diego

VGG16 trained on ImageNet data is used as a pre-trained model from which to extract features. Features are then passed through fully connected layers to classify cats vs. dogs.

Adapted from tensorflow tutorials
(https://github.com/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning.ipynb)

---


## Setup

In [ ]:
import tensorflow as tf
from tensorflow.compat.v1.keras import backend as K
from tensorflow.keras import applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img

from sklearn.metrics import classification_report 
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import time

In [ ]:
print("Tensorflow version:",tf.__version__)
!python --version
print(tf.config.list_physical_devices('GPU'))
!nvidia-smi

In [ ]:
# Set logging level
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

In [ ]:
# Set random generator seed
seed = 1234

# Disable hash randomization by specifying the value 0.
os.environ['PYTHONHASHSEED'] = '0'

# Set numpy random generator
np.random.seed(seed)

# Set python built-in random generator
random.seed(seed)

# Set tf global random seed
tf.random.set_seed(seed)

# Set tensorflow graph-level random seed
tf.compat.v1.random.set_random_seed(seed)

# Potential randomness from CUDNN
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC']= '1'

### Set image dimensions, location of images

In [ ]:
# Image dimensions 
img_width, img_height = <<FILL-IN>> , <<FILL-IN>> 
IMG_SIZE = (img_width,img_height) #image size (150,150)
IMG_SHAPE = IMG_SIZE + (<<FILL-IN>> ,) #image shape(3,150,150)

# Location of images
data_dir = os.getenv('HOME') + '/data/ml/catsVsDogs/'
train_data_dir      = data_dir + <<FILL-IN>>
validation_data_dir = data_dir + <<FILL-IN>>
test_data_dir       = data_dir + <<FILL-IN>>

# Batch size
BATCH_SIZE = <<FILL-IN>>

### Prepare data

In [ ]:
# Data augmentation setup
datagen = ImageDataGenerator(rescale = 1. / 255)

# Set up generator to read images found in subfolders of training data directory,
# and indefinitely generate batches of image data (scaled).  This is for training data.
train_generator = datagen.flow_from_directory(train_data_dir,target_size=<<FILL-IN>>, # img size we defined above
                                              batch_size = <<FILL-IN>> , class_mode='binary', # batch size we defined above 
                                              shuffle = True, seed = <<FILL-IN>>)   # seed we defined above           

# Set up generator to generate batched of validation data for model
validation_generator = datagen.flow_from_directory(<<FILL-IN>>,target_size=<<FILL-IN>>,
                                                   batch_size = <<FILL-IN>>,class_mode=<<FILL-IN>>,
                                                   shuffle = <<FILL-IN>>, seed = <<FILL-IN>>) # no need to shuffle val data
# Set up generator to generate batched of test data for model
test_generator = datagen.flow_from_directory(<<FILL-IN>>, target_size=<<FILL-IN>>,
                                             batch_size = <<FILL-IN>>, class_mode=<<FILL-IN>>,
                                             shuffle = <<FILL-IN>>, seed = <<FILL-IN>>) # no need to shuffle test data

### Load pre-trained VGG16 model

In [ ]:
# Load VGG16 model's Imagenet weights not including the top fully connected layers

base_model = applications.VGG16(include_top = <<FILL-IN>>, # don't include top https://www.tensorflow.org/api_docs/python/tf/keras/applications/vgg16/VGG16
                                weights = <<FILL-IN>>, #use Imagenet weights
                                input_shape=IMG_SHAPE) 
# Freeze all weights of VGG16 model
base_model.trainable = <<FILL-IN>>
base_model.summary()

### Create top model

Create fully connected layers to put on top of pre-trained VGG16 

In [ ]:
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(<<FILL-IN>>, activation=<<FILL-IN>>)) #add Dense layer with 256 units and relu activation https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dense
model.add(Dropout(<<FILL-IN>>)) #apply 50% dropout rate. https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout
model.add(Dense(1, activation='sigmoid'))
model.summary()

### Train model
Train weights in top model

In [ ]:
EPOCHS = 20

# Compile model with Adam optimizer with slow learning rate,
# Binary Cross-Entropy loss function and Accuracy metric
model.compile(optimizer= <<FILL-IN>>, #use Adam optimizer with 0.0001 learning rate https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam
              loss= <<FILL-IN>>, #use default Binary Cross-entropy loss https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy
              metrics=[<<FILL-IN>>]) #use accuracy metrics; this will be printed out during training in each epochs https://www.tensorflow.org/api_docs/python/tf/keras/metrics/Accuracy

# Early Stopping to avoid overfitting and ModelCheckpoint to save the best model
checkpoint_path = 'tmp/checkpoint'
callbacks = [EarlyStopping(monitor=<<FILL-IN>>,patience=3,min_delta=0.001, #use val_loss to monitor during training https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping
                           mode=<<FILL-IN>>), # we want to minimize val_loss
             ModelCheckpoint(filepath=checkpoint_path, monitor='val_loss',
                             mode = 'min', save_best_only = True, 
                             save_weights_only=True)]

In [ ]:
%%time 

train_history = model.<<FILL-IN>>(train_generator,epochs=EPOCHS, #train the model using model.fit()
                          validation_data=validation_generator, 
                          callbacks=callbacks)

In [ ]:
# Load the best model that was saved using ModelCheckpoint
model.load_weights(checkpoint_path)

In [ ]:
# Save model weights 
model.save("weights_from_feature_extract")

In [ ]:
# Plot train and validation history
fig, axs = plt.subplots(1,2, figsize= (20,5))
axs[0].plot(train_history.history['loss'])
axs[0].plot(train_history.history['val_loss'])
axs[0].set_title("Train, Val loss history")
axs[0].set_xlabel("Epoch")
axs[0].legend(["Train Loss","Val Loss"])

axs[1].plot(train_history.history['accuracy'])
axs[1].plot(train_history.history['val_accuracy'])
axs[1].set_title("Train, Val Accuracy history")
axs[1].set_xlabel("Epoch")
axs[1].legend(["Train Accuracy","Val Accuracy"])

### Evaluate model

In [ ]:
_, train_accuracy = model.evaluate(train_generator)
print("Train data accuracy:", train_accuracy)

_, test_accuracy = <<FILL-IN>> ## evaluate test data
print("Test data accuracy:", test_accuracy)

In [ ]:
# Get predicted value and the ground truth value of test data
pred = (model.predict(test_generator) > 0.5).astype("int32")
true = test_generator.classes

In [ ]:
# Classification report for test data
print(classification_report(y_true= true, y_pred = pred, target_names=['cats', 'dogs']))

### Perform inference on test images

In [ ]:
def image_loader(img_file):
    img = load_img(img_file, target_size = (img_width, img_height))
    imgplot = plt.imshow(img)
    plt.show()
    img = img_to_array(img) / 255
    img = np.expand_dims(img, axis = 0) #model input is (1,150,150,3)
    return img

In [ ]:
img = image_loader(data_dir + 'test/cats/cat.1070.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
img = image_loader(data_dir + 'test/dogs/dog.1233.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)

In [ ]:
img = image_loader(data_dir + 'test/cats/cat.1195.jpg')
img_y_pred = model.predict(img) 
print(img_y_pred)